In [1]:
# %load std_libraries.py
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\\heine\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)

import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set(style="ticks", color_codes=True)
#from sklearn import linear_model
#import pickle
#import subprocess
#import networkx as nx
# import scipy.stats as stats
import scipy.cluster.hierarchy as sch

from Bio import SeqIO
from Bio import SeqFeature as sf
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq

import re

from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02_student','9dMTMZgJMgUP0YX0P5mQ')
    #py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')


what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
Added C:\Users\heine\github\expression_broad_data to path: 
['', 'C:\\Users\\heine\\Anaconda3\\python36.zip', 'C:\\Users\\heine\\Anaconda3\\DLLs', 'C:\\Users\\heine\\Anaconda3\\lib', 'C:\\Users\\heine\\Anaconda3', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\Babel-2.5.0-py3.6.egg', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\heine\\.ipython', 'C:\\Users\\heine\\github\\expression_broad_data']
I am about to import a library
I am importing io_library
what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
data process

## Load Data

In [4]:
#Read in the KL promoter database.  
kl_promoters = pd.read_pickle(data_processing_dir + os.path.normpath("kl_promoters/kl_promoters.pkl"))

#Read in the SC promoter database.  
sc_promoters = pd.read_pickle(data_processing_dir + os.path.normpath("sc_promoters/sc_promoters.pkl"))

In [3]:
# Load goi data from goi data file created in paralog analysis script. 
goi_data = pd.read_csv(data_processing_dir + os.sep + "20170817_klscpka" + os.sep + "goi_data.csv")     #_solis.csv")

## Build enrichment sets

Did enrichment analysis using DREME, MEME, and GLAM2 on the upregulated set.  

Only found Msn2 and a few others 


In [21]:
a = '2.0'

In [27]:
#Upregulated v.s. all
LFC_cutoff_high = 2.0
genes = goi_data[goi_data['log2FoldChange_SC']>LFC_cutoff_high]['sc_genename']

target_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/paralogs_LFC_gt_' + str(LFC_cutoff_high).replace('.','p') + '.fasta')

io_library.write_promoter_file(sc_promoters, list(genes),target_promoter_fname)

#Downregulated v.s. all
LFC_cutoff_low = -2.0
genes = goi_data[goi_data['log2FoldChange_SC']<LFC_cutoff_high]['sc_genename']

target_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/paralogs_LFC_lt_' + str(LFC_cutoff_low).replace('-','m').replace('.','p') + '.fasta')

io_library.write_promoter_file(sc_promoters, list(genes),target_promoter_fname)



#Upregulated v.s. flat and downregulated
#downregulated v.s. flat and upregulated

all_sc_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/all_sc_promoters.fasta')

YIL111W not in promoter data set.


## Run MEME to look for unbiased enrichment

In [ ]:
#Right now meme doesn't do discriminatory and allow any number of motifs to be scanned. 
#Dreme seems to be better for that. 
#Ran GLAM2 and got a bunch of polyA sites. 


#run psp-gen to get prior for positive and negative sequences. 
psp_gen_command = ['psp-gen', '-pos', target_promoter_fname, '-neg', all_sc_promoter_fname, '-dna', 
                   '-minw', '6', #minimum width of motif
                   '-maxw', '20', #maximum width of motif
                   '>', 'psp_prior.psp']
subprocess.run(psp_gen_command)

In [ ]:
meme paralogs_LFC_gt_2p0.fasta -dna -oc . -nostatus -time 17358 -maxsize 60000 -mod anr -nmotifs 5 -minw 6 -maxw 50 -revcomp -psp priors.psp
meme paralogs_LFC_gt_2p0.fasta -dna -oc . -nostatus -time 17402 -maxsize 60000 -mod zoops -nmotifs 3 -minw 6 -maxw 50 -revcomp -psp priors.psp
dreme dreme -v 1 -oc . -dna -p paralogs_LFC_gt_2p0.fasta -n all_sc_promoters.fasta -t 18000 -e 0.05

## Look for Msn2 motifs 

In [5]:
#Motif Prior from Roy et al: 

#HSP12: YFL014W
#Msn2: YMR037C
motif_prior = pd.read_table(data_processing_dir + os.path.normpath("/motifs/Motif_prior.txt"), header = None, names = ["TF","sc_genename","interaction_score"])

#shrink motif_prior to include only genes that are included on my subset list
motif_prior_gois = motif_prior[motif_prior["sc_genename"].isin(goi_data['sc_genename'])]
goi_data_msn2binding = goi_data.copy()
#goi_data_msn2binding.sort_values('log2FoldChange_msn24del_interaction', axis = 0, inplace=True)

for tf in ['MSN2', 'MSN4']:
    tf_hits = motif_prior_gois[motif_prior_gois['TF'].isin(io_library.SC_orf_lookup_by_name([tf]))]
    goi_data_msn2binding = goi_data_msn2binding.merge(tf_hits.loc[:,['sc_genename','interaction_score']],how = 'left', on = 'sc_genename')
    #Replace NAN's
    goi_data_msn2binding['interaction_score'] = goi_data_msn2binding['interaction_score'].fillna(0)
    goi_data_msn2binding.rename(columns = {'interaction_score': tf + '_roy_score'}, inplace = True)





In [5]:
#generate promoter file for all gois: 
genes = goi_data_msn2binding['sc_genename']
target_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/goi.fasta')
io_library.write_promoter_file(sc_promoters, list(genes),target_promoter_fname)



In [ ]:
#Run FIMO for particular motifs
#ran fimo using webtool 20180216
#fimo --oc . --verbosity 1 --thresh 0.001 msn2_all.meme goi.fasta



In [6]:
#Load the fimo search data 
fimo_fname = os.path.normpath(data_processing_dir + 'sc_promoters' + os.sep + 'fimo_output' + os.sep + '20180216_goi_msn2_all.txt')
fimo_hits = pd.read_table(fimo_fname)
    
motifs = list(set(fimo_hits['# motif_id']))

for motif in motifs: 
    fimo_hits_motif = fimo_hits[fimo_hits['# motif_id']==motif]
    hit_dict = {}
    for ind, row in fimo_hits_motif.iterrows():
        gene = row['sequence_name']
        loc = 700-row['start']
        pval = row['p-value']
        seq = row['matched_sequence']
        if gene in hit_dict.keys(): 
            prev_hits = hit_dict[gene]
            prev_hits.append((loc,pval,seq))
        else: 
            hit_dict[gene] = [(loc,pval, seq)]

    hit_column = []
    for gene in goi_data_msn2binding['sc_genename']: 
        if gene in hit_dict.keys():
            hit_column.append(hit_dict[gene])
        else:
            hit_column.append(None)

    goi_data_msn2binding[motif] = hit_column


In [7]:
#Run exact motif matches

#Searches for exact matches of defined motifs in various sequences
motif_dict = {'STRE':'AGGGG',
             'Msn2_pref': 'CGGGG' 
             }
promoter_database = sc_promoters.loc[goi_data_msn2binding['sc_genename'],:]
gene_list = goi_data_msn2binding['sc_genename']
output_format = 'full'
sequence_context = 2
promoter_counts = io_library.exact_promoter_scan(gene_list, motif_dict, promoter_database, output_format, sequence_context)

goi_data_msn2binding = goi_data_msn2binding.merge(promoter_counts,left_on='sc_genename',right_index=True,how='left')



In [8]:
goi_data_msn2binding.to_csv(data_processing_dir + '20170817_klscpka/goi_data_msn2_binding.csv')

In [81]:
gene_list = ['COX5A','COX5B', 'TKL1','TKL2', 'GDH1','GDH3','CIT1','CIT2','GND1','GND2','MPC2','MPC3','CDC19','PYK2','SOL3','SOL4','TAL1','NQM1']
columns_of_interest = ['SC_common_name','log2FoldChange_SC','MSN2_roy_score','MSN4_roy_score', 'STRE', 'Msn2_pref'] + motifs
goi_msn2_subset = goi_data_msn2binding[goi_data_msn2binding['SC_common_name'].isin(gene_list)].loc[:,columns_of_interest]
goi_msn2_subset.to_csv(data_processing_dir + 'sc_promoters/goi_subset_msn2_binding.csv')

In [51]:
fc_values = goi_data_msn2binding.loc[:,['log2FoldChange_SC','log2FoldChange_msn24del_interaction']]
fc_names = ['LFC','LFC Msn24 int']

msn2_promoter_binding_columns = ['MSN2_roy_score', 'MSN4_roy_score']
msn2_promoter_binding_names = ['MSN2_roy', 'MSN4_roy']
msn2_promoter_binding_values =goi_data_msn2binding.loc[:,msn2_promoter_binding_columns]

#want this label to capture paralog pairs eventually
gene_labels = list(goi_data_msn2binding['SC_common_name'])

trace1 = pygo.Heatmap(z = fc_values.values,
                   x= fc_names,
                   y= gene_labels, 
                   showscale = False
                   )

trace2 = pygo.Heatmap(z = msn2_promoter_binding_values.values,
                   #   zmin = -3.0, 
                   #   zmax = 3.0,
                   #text = np.array(joint_topology_text).transpose(),
                   x= msn2_promoter_binding_names,
                   y= gene_labels, 
                   #hoverinfo = 'x+y+text',
                   colorscale = 'Electric',
                   showscale = False)

# trace3 = pygo.Heatmap(z = growth_phenotypes.values,
#                       #text = np.array(topology_text).transpose(),
#                    x=growth_column_names,
#                    y= gene_labels,
#                    colorscale = 'Blackbody',
#                    #hoverinfo = 'x+y+text',
#                    showscale = False)


fig = pytools.make_subplots(rows=1, cols=2)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
#fig.append_trace(trace3, 1, 3)

#fig['layout'].update(height = 2000)
#fig['layout'].update(height=600, width=600, title='i <3 subplots')

# layout = pygo.Layout(
#     yaxis = {'title': 'Low Paralog / High Paralog'}
# )

#fig = pygo.Figure(data=data, layout=layout)
py.iplot(fig, filename='goi_msn2_heatmap')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [13]:
goi_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'SC_common_name', 'sc_genename',
       'log2FoldChange_SC', 'raw_AS_noNMPP1_YPD_t40_mean_SC', 'pvalue_SC',
       'raw_AS_yesNMPP1_YPD_t40_mean_SC', 'kl_genename', 'log2FoldChange_KL',
       'raw_AS_noNMPP1_YPD_t40_mean_KL', 'raw_AS_yesNMPP1_YPD_t40_mean_KL',
       'pvalue_KL', 'category', 'log2FoldChange_msn24del_interaction',
       'pvalue_msn24del_interaction'],
      dtype='object')

In [ ]:
io_library.SC_common_name_lookup([])